In [15]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

was trying an amazon sagemaker just to get more compute power for training the model. this is left over code from that.

In [16]:
# import zipfile
# import os

# def unzip_file(zip_file_path):
#     # Get the directory where the zip file is located
#     extract_to_directory = os.path.splitext(zip_file_path)[0]

#     with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#         zip_ref.extractall(extract_to_directory)

# notebook_directory = os.path.dirname(os.path.abspath("main.ipynb"))

# # Replace 'Kindle_Store.zip' with the actual name of your zip file
# zip_file_name = 'Kindle_Store.zip'

# # Full path to the zip file
# zip_file_path = os.path.join(notebook_directory, zip_file_name)
# print(zip_file_path)
# # Check if the zip file exists
# if os.path.exists(zip_file_path):
#     # Call the function to unzip the file
#     unzip_file(zip_file_path)
# else:
#     print("The specified zip file does not exist.")


## Loading the data using chunking

In [17]:
# Read JSON file in chunks
chunk_size = 100000  # Adjust the chunk size as needed
chunks = []
with open('Kindle_Store.json', 'r') as file:
    for chunk in pd.read_json(file, chunksize=chunk_size, lines=True):
        chunks.append(chunk)

# Concatenate all chunks to form a single DataFrame
df = pd.concat(chunks, ignore_index=True)

In [18]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4,True,"12 29, 2012",A27UD5HYAKBL97,1423600150,{'Format:': ' Hardcover'},Cheryl,If you like making salsas this is a great book...,Great Book,1356739200,NaN,NaN
1,5,True,"03 6, 2012",A8P5DK8LLOYGH,1423600150,{'Format:': ' Hardcover'},Shay365,great little book. simple and right to the poi...,great little book,1330992000,3,NaN
2,5,True,"08 12, 2009",A3OM9W7DXSUIIY,1423600150,{'Format:': ' Hardcover'},R. Peckham,This book has good pics of the recipes and eas...,very good bok with good ideas.,1250035200,NaN,NaN
3,3,False,"11 3, 2006",A3DPAR2PWB9BT8,1423600150,{'Format:': ' Hardcover'},JJSS,"I admire the SF School of Cooking, but was dis...",Ok cookbook,1162512000,3,NaN
4,5,False,"04 11, 2006",A2ONTBSAX7GIOV,1423600150,{'Format:': ' Hardcover'},Three Guys from Miami,Many people are surprised to find out that we ...,Fresh and Tasty Ideas from Santa Fe!,1144713600,15,NaN


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5722988 entries, 0 to 5722987
Data columns (total 12 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   overall         int64 
 1   verified        bool  
 2   reviewTime      object
 3   reviewerID      object
 4   asin            object
 5   style           object
 6   reviewerName    object
 7   reviewText      object
 8   summary         object
 9   unixReviewTime  int64 
 10  vote            object
 11  image           object
dtypes: bool(1), int64(2), object(9)
memory usage: 485.8+ MB


turn image into dummy

In [20]:
# if image is non null return 1 else 0
df['image'] = df['image'].notnull().astype('int')

convert votes into int

In [21]:
# remove commas from vote column
df['vote'] = df['vote'].str.replace(',', '')
# cast vote column to int, converting Nan to 0
df['vote'] = df['vote'].fillna(0)
df['vote'] = df['vote'].astype('int')

replace null with empty string

In [22]:
# in review text replace null values with empty string
df['reviewText'] = df['reviewText'].fillna('')

convert verfied to dummy

In [23]:
# convert verified to int 1 if true 0 if false
df['verified'] = df['verified'].astype('int')

checking the types of styles

In [27]:
df['style'].value_counts()

{'Format:': ' Kindle Edition'}    5045160
Name: style, dtype: int64

making sure we are only looking at kindle reviews

In [28]:
# drop row if style is not {'Format:': ' Kindle Edition'} 
df = df[df['style'] == {'Format:': ' Kindle Edition'}]
# drop style column
df = df.drop(columns=['style'])

dropped reviewtime, already have unix representation

In [30]:
# drop the reviewTime column
df = df.drop(columns='reviewTime')

creating review length column

In [32]:
# create a new column called reviewLength
df['reviewLength'] = df['reviewText'].str.len()

creating summary length column

In [33]:
# create a new column called summaryLength
df['summaryLength'] = df['summary'].str.len()

asin is the amazon product id. removing this and the reviewer info

In [36]:
# removing asin, reviewerID, reviewerName
df = df.drop(columns=['asin', 'reviewerID', 'reviewerName'])

labeling categorical variables

In [39]:
# labeling ['verified', 'image'] as categorical
df[['verified', 'image']] = df[['verified', 'image']].astype('category')

reviewing data

In [40]:
df.head()

,overall,verified,reviewText,summary,unixReviewTime,vote,image,reviewLength,summaryLength
9,5,1,For the salsa lover who likes new recipes!!,Five Stars,1418688000,0,0,43,10.0
10,1,1,Recipes are good but the kindle book is just t...,Very bad look in old black and white photocopi...,1414368000,0,0,326,54.0
14,1,1,No color photos. Would not recommend to others...,Not in color as portrayed on Amazon site,1367884800,0,0,130,40.0
15,3,1,"Enjoyed the read. A bit of a twist to it, fun ...",western with a twist.,1412985600,0,0,143,21.0
16,5,1,I have enjoyed all work by this author,Five Stars,1412467200,0,0,38,10.0


reviewing data types

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5045160 entries, 9 to 5722987
Data columns (total 9 columns):
 #   Column          Dtype   
---  ------          -----   
 0   overall         int64   
 1   verified        category
 2   reviewText      object  
 3   summary         object  
 4   unixReviewTime  int64   
 5   vote            int32   
 6   image           category
 7   reviewLength    int64   
 8   summaryLength   float64 
dtypes: category(2), float64(1), int32(1), int64(3), object(2)
memory usage: 427.3+ MB


## Creating BOW

In [20]:
vectorizer = CountVectorizer(stop_words='english')
reviews = df['reviewText'].tolist()
fitted = vectorizer.fit(reviews)
bow = fitted.transform(reviews)

in the future could make test_size bigger. figure dataset is so big that it doesn't matter.

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Step 1: Prepare the data
X = bow  # Your Bag-of-Words representation
y = df['overall']  # Target variable

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

In [22]:
# code from a friends notebook that allows for training in batches
# import numpy as np
# import tqdm
# def pred_pairs(alg, test_data):
#     results = []
#     for i in tqdm(range(0, test_data.shape[0], chunk_size)):
#         batch = test_data[i:i+chunk_size]
#         batch_preds = alg.predict(batch)
#         rows, cols = batch.nonzero()
#         batch_labels_nonzero = np.ravel(batch[rows, cols]).tolist()
#         batch_preds_nonzero = np.ravel(batch_preds[rows, cols]).tolist()
#         y_label = batch_labels_nonzero
#         y_pred = batch_preds_nonzero
#         for y_l, y_p in zip(y_label, y_pred):
#             results.append([y_l, y_p])
#     return np.array(results)

to do log regression makes sparse array dense. this is a problem for large datasets.

In [23]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler(with_mean=False).fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
# import linear regression
from sklearn.linear_model import LinearRegression
# Step 3: Train a logistic regression model
log_reg = LinearRegression()
log_reg.fit(X_train, y_train)


LinearRegression()

In [25]:
# Step 4: Evaluate the model
accuracy = log_reg.score(X_test, y_test)
print(f"Accuracy of the linear regression model: {accuracy:.2f}")

Accuracy of the linear regression model: 0.14


### Lemmatization

lemmatization is the process of grouping together the different inflected forms of a word so they can be analysed as a single item. Lemmatization is similar to stemming but it brings context to the words. So it links words with similar meaning to one word.

the count vectorizer uses stemming by default, which is a rudimentary heuristic process that chops off the ends of words. this is not ideal for our use case, so we might use lemmatization instead.

In [26]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer(),
                                strip_accents='unicode', 
                                stop_words='english',
                                max_df=0.5,
                                min_df=10)
# ignores the paremter if it occurs in more than 50% of the documents
# ignores the parameter if it occurs in less than 10 documents
tf_vectorizer.fit(reviews)
tf = tf_vectorizer.transform(reviews)

c:\Users\youngkj\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\youngkj\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
